## Streams Examples

### Different kind of streams

Besides regular object streams Java8 ships with special kinds of streams for working with the primitive data types <code>int</code>, <code>long</code> and <code>double</code> such as <code>IntStream</code>, <code>LongStream</code> and <code>DoubleStream</code>.

<code>IntStream</code> can replace the regular for-loop utilizing <code>IntStream.range()</code>:

In [1]:
import java.util.stream.IntStream;
import java.util.stream.Stream;
import java.util.function.Supplier;

In [2]:
IntStream.range(1, 4)
    .forEach(System.out::println);

1
2
3


All those primitive streams work just like regular object streams with the following differences: Primitive streams use specialized lambda expressions, e.g. <code>IntFunction</code> instead of <code>Function</code> or <code>IntPredicate</code> instead of <code>Predicate</code>. And primitive streams support the additional terminal aggregate operations <code>sum()</code> and <code>average()</code>:

In [3]:
Arrays.stream(new int[]{1, 2, 3})
    .map(i -> 2 * i + 1)
    .average()
    .ifPresent(System.out::println);

5.0


Sometimes it's useful to transform a regular object stream to a primitive stream or vice versa. For that purpose object streams support the special mapping operations <code>mapToInt()</code>, <code>mapToLong()</code> and <code>mapToDouble</code>:

In [4]:
Stream.of("a1", "a2", "a3")
    .map(s -> s.substring(1))
    .mapToInt(Integer::parseInt)
    .max()
    .ifPresent(System.out::println);

3


Primitive streams can be transformed to object streams via <code>mapToObj()</code>:

In [5]:
IntStream.range(1, 4)
    .mapToObj(i -> "a" + i)
    .forEach(System.out::println);

a1
a2
a3


Combined example: the stream of doubles is first mapped to an int stream and than mapped to an object stream of strings:

In [6]:
Stream.of(1.0, 2.0, 3.0)
    .mapToInt(Double::intValue)
    .mapToObj(d -> "a" + d)
    .forEach(System.out::println);

a1
a2
a3


### Processing Order

An important characteristic of <i>intermediate</i> operations is laziness. In the next sample a <i>terminal</i> operation is missing:

In [7]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .filter(s -> {
        System.out.println("filter: " + s);
        return true;
    });

java.util.stream.ReferencePipeline$2@344e9e86

Nothing is printed to the console. That is because <i>intermediate</i> operations will only be executed when a <i>terminal</i> operation is present.

If we add for example <i>terminal</i> operation <code>forEach</code> the result of the <code>filter</code> operation will be outputted on the console:

In [8]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .filter(s -> {
        System.out.println("filter: " + s);
        return true;
    })
    .forEach(s -> System.out.println("forEach: " + s));

filter: d2
forEach: d2
filter: a2
forEach: a2
filter: b1
forEach: b1
filter: b3
forEach: b3
filter: c
forEach: c


As we can see the order of execution is - The first string "d2" passes filter then forEach, only then the second string "a2" is processed and so on.

This behavior can reduce the actual number of operations performed on each element:

In [9]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .map(s -> {
        System.out.println("map: " + s);
        return s.toUpperCase();
    })
    .anyMatch(s -> {
        System.out.println("anyMatch: " + s);
        return s.startsWith("A");
    });

map: d2
anyMatch: D2
map: a2
anyMatch: A2


true

The operation <code>anyMatch</code> returns <code>true</code> as soon as the predicate applies to the given input element. This is <code>true</code> for the second element passed "A2". Due to the vertical execution of the stream chain, map has only to be executed twice in this case. So instead of mapping all elements of the stream, map will be called as few as possible.

#### Why order of operations matters

In the next example both <code>map</code> and <code>filter</code> are called five times for every string in the underlying collection whereas forEach is only called once:

In [10]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .map(s -> {
        System.out.println("map: " + s);
        return s.toUpperCase();
    })
    .filter(s -> {
        System.out.println("filter: " + s);
        return s.startsWith("A");
    })
    .forEach(s -> System.out.println("forEach: " + s));

map: d2
filter: D2
map: a2
filter: A2
forEach: A2
map: b1
filter: B1
map: b3
filter: B3
map: c
filter: C


If change the order of the operations, moving <code>filter</code> to the beginning of the chain, the result will be another:

In [11]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .filter(s -> {
        System.out.println("filter: " + s);
        return s.startsWith("a");
    })
    .map(s -> {
        System.out.println("map: " + s);
        return s.toUpperCase();
    })
    .forEach(s -> System.out.println("forEach: " + s));

filter: d2
filter: a2
map: a2
forEach: A2
filter: b1
filter: b3
filter: c


<code>map</code> is only called once. So the operation pipeline performs much faster for larger numbers of input elements.

In [12]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .sorted((s1, s2) -> {
        System.out.printf("sort: %s; %s\n", s1, s2);
        return s1.compareTo(s2);
    })
    .filter(s -> {
        System.out.println("filter: " + s);
        return s.startsWith("a");
    })
    .map(s -> {
        System.out.println("map: " + s);
        return s.toUpperCase();
    })
    .forEach(s -> System.out.println("forEach: " + s));

sort: a2; d2
sort: b1; a2
sort: b1; d2
sort: b1; a2
sort: b3; b1
sort: b3; d2
sort: c; b3
sort: c; d2
filter: a2
map: a2
forEach: A2
filter: b1
filter: b3
filter: c
filter: d2


Sorting is a special kind of <i>intermediate</i> operation. It's a so called stateful operation since in order to sort a collection of elements you have to maintain state during ordering.

The <code>sort</code> operation is executed on the entire input collection. In other words sorted is executed <i>horizontally</i>. So in this case sorted is called eight times for multiple combinations on every element in the input collection.

We can optimize the performance by reordering the chain:

In [13]:
Stream.of("d2", "a2", "b1", "b3", "c")
    .filter(s -> {
        System.out.println("filter: " + s);
        return s.startsWith("a");
    })
    .sorted((s1, s2) -> {
        System.out.printf("sort: %s; %s\n", s1, s2);
        return s1.compareTo(s2);
    })
    .map(s -> {
        System.out.println("map: " + s);
        return s.toUpperCase();
    })
    .forEach(s -> System.out.println("forEach: " + s));

filter: d2
filter: a2
filter: b1
filter: b3
filter: c
map: a2
forEach: A2


<code>sorted</code> is never been called because <code>filter</code> reduces the input collection to just one element. So the performance is greatly increased for larger input collections.

### Reusing Streams

Java8 streams cannot be reused. As soon as you call any terminal operation the stream is closed:

In [14]:
Stream<String> stream =
    Stream.of("d2", "a2", "b1", "b3", "c")
        .filter(s -> s.startsWith("a"));

stream.anyMatch(s -> true);    // ok
stream.noneMatch(s -> true);   // exception

EvalException: stream has already been operated upon or closed

To overcome this limitation we have to to create a new stream chain for every terminal operation we want to execute, e.g. we could create a stream supplier to construct a new stream with all intermediate operations already set up:

In [15]:
Supplier<Stream<String>> streamSupplier = 
    () -> Stream.of("d2", "a2", "b1", "b3", "c")
        .filter(s -> s.startsWith("a"));

In [16]:
streamSupplier.get().anyMatch(s -> true);
streamSupplier.get().noneMatch(s -> true);

false

Each call to <code>get()</code> constructs a new stream on which we are save to call the desired terminal operation.